# Статистика, DS-поток
## Практическое задание 9


**Правила:**

* Дедлайн **9 ноября 16:00**. После дедлайна работы не принимаются кроме случаев наличия уважительной причины.
* Выполненную работу нужно отправить телеграм-боту `@miptstats_ad21_bot`.
* Прислать нужно ноутбук в формате `ipynb`, а также файл `utils.py` с вашим кодом.
* Решения, размещенные на каких-либо интернет-ресурсах не принимаются. Публикация решения может быть приравнена к предоставлении возможности списать.
* Для выполнения задания используйте этот ноутбук в качествие основы, ничего не удаляя из него.
* Никакой код из данного задания при проверке запускаться не будет.

**Баллы за задание:**

* Задача 1 &mdash; 6 баллов;
* Задача 2 &mdash; 3 балла;
* Задача 3 &mdash; 2 балла;
* Задача 4 &mdash; 20 баллов.

**Важность задач:**

* *высокая:* задачи 1-3;
* *средняя:* задача 4.

In [118]:
import numpy as np
import pandas as pd
import scipy.stats as sps
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as pof

from statsmodels.sandbox.stats.multicomp import multipletests

np.set_printoptions(precision=3)
pd.set_option("precision", 3)

_____
## Задача 3.
Были проведены эксперименты для оценки эффективности нескольких препаратов для снижения послеоперационной тошноты. Результаты экспериментов приведены в таблице ниже. При проведении эксперимента пациенты делились на группы случайным образом.




|| Количество пациентов   | Количество случаев возникновения тошноты |
|-----------| ----------- | -------|
|Плацебо|80|45|
|Хлорпромазин|75|26|
|Дименгидринат|85|52|
|Пентобарбитал (100 мг)|67|35|
|Пентобарбитал (150 мг)|85|37|




Проведите сравнение каждого препарата по эффективности по отношению к плацебо. Какие ответы можно получить для методов, контролирующих FWER и FDR? В каждом случае приведите подправленные p-value.

*Замечание.* Используйте [`multipletests`](https://www.statsmodels.org/dev/generated/statsmodels.stats.multitest.multipletests.html) из библиотеки `statsmodels`. 

**Решение:** посчитаем частоту случаев возникновения тошноты после плацебо: $p_0 = \frac{45}{80} = 0.5625.$ Рассмотрим гипотезы вида: $H_0^i: p_i = p_0 \ vs. H_1^i: p_i < p_0,$ проще говоря, для каждого препарата проведем AB-тест с плацебо и выясним, какой препарат эффективнее плацебо, а какой нет. <br>
Из лекции критерий Вальда для AB-теста с гипотезами $H_0^i: p_i = p_0 \ vs. H_1^i: p_i < p_0: \{S = W(x, y) > z_{1 - \alpha}\},$ где $W(X, Y) = \frac{\widehat{p_0} - \widehat{p_i}}{\widehat{\sigma}}, \ \widehat{p_j} = \frac{m_j}{n_j},$ где $m_j$ &mdash; количество единиц в выборке $Y$ ($X$, если $j = 0$) размера $n_j.$ <br>
$$\widehat{\sigma} = \sqrt{\frac{\widehat{p_0}(1 - \widehat{p_0})}{n_0} + \frac{\widehat{p_i}(1 - \widehat{p_i})}{n_i}}.$$
Тогда рассчитаем p-value: $pval_i(w) = P(W(x, y) > w) = $ `sps.norm.sf`($w$).

In [47]:
n_0 = 80
m_0 = 45
n = np.array([75, 85, 67, 85])
m = np.array([26, 52, 35, 37])
p_0 = m_0 / n_0
p = m / n
alpha = 0.05

In [48]:
def AB_test(p_0, p, method='bonferroni'):
    sigma = np.sqrt((p_0 * (1 - p_0)) / n_0 + (p * (1 - p)) / n)
    w = (p_0 - p) / sigma
    pvals = sps.norm.sf(w)
    reject, pvalscorr = multipletests(pvals=pvals, alpha=alpha, method=method, is_sorted=False)[:2]
    return pvals, reject, pvalscorr

In [49]:
def make_table(cols, indexes, n, m, pvals, pvalscorr, reject):
    d = {cols[0]: n, cols[1]: m, cols[2]: pvals, cols[3]: pvalscorr, cols[4]: reject}
    data = pd.DataFrame(d)
    indexes = pd.Index(indexes)
    data = data.set_index(indexes)
    return data

In [50]:
cols = np.array(['Количество пациентов', 'Количество случаев возникнвения тошносты', 'p-value', 
                 'Скорректированное p-value', 'Лучше, чем плацебо'])
indexes = np.array(['Хлорпромазин', 'Дименгидринат', 'Пентобарбитал (100 мг)', 'Пентобарбитал (150 мг)'])

Теперь проверим работу разных методов. Заметим, что методы Шидака, Шидака-Холма и Бенджамини-Хохберга мы применять не можем, так как критерии не независимы, поэтому рассмотрим работу методов Бонферрони, Холма и Бенджамини-Иекутиели.

Метод Бонферрони.

In [51]:
pvals, reject, pvalscorr = AB_test(p_0, p, 'bonferroni')
make_table(cols, indexes, n, m, pvals, pvalscorr, reject)

,Количество пациентов,Количество случаев возникнвения тошносты,p-value,Скорректированное p-value,"Лучше, чем плацебо"
Хлорпромазин,75,26,0.003,0.011,True
Дименгидринат,85,52,0.740,1.000,False
Пентобарбитал (100 мг),67,35,0.313,1.000,False
Пентобарбитал (150 мг),85,37,0.050,0.199,False


Метод Холма.

In [52]:
pvals, reject, pvalscorr = AB_test(p_0, p, 'holm')
make_table(cols, indexes, n, m, pvals, pvalscorr, reject)

,Количество пациентов,Количество случаев возникнвения тошносты,p-value,Скорректированное p-value,"Лучше, чем плацебо"
Хлорпромазин,75,26,0.003,0.011,True
Дименгидринат,85,52,0.740,0.740,False
Пентобарбитал (100 мг),67,35,0.313,0.627,False
Пентобарбитал (150 мг),85,37,0.050,0.149,False


Метод Бенджамини-Иекутиели.

In [53]:
pvals, reject, pvalscorr = AB_test(p_0, p, 'fdr_by')
make_table(cols, indexes, n, m, pvals, pvalscorr, reject)

,Количество пациентов,Количество случаев возникнвения тошносты,p-value,Скорректированное p-value,"Лучше, чем плацебо"
Хлорпромазин,75,26,0.003,0.024,True
Дименгидринат,85,52,0.740,1.000,False
Пентобарбитал (100 мг),67,35,0.313,0.870,False
Пентобарбитал (150 мг),85,37,0.050,0.208,False


**Вывод:** в данном примере мы видим одинаковый результат касаемо того, лучше ли препарат, чем плацебо, или нет у всех рассмотренных методов множественной проверки гипотез: лучше, чем плацебо, оказался только препарат `Хлорпромазин` (стоит заметить, однако, что мы не утверждаем, что остальные препараты хуже или не лучше: мы лишь только не можем определенно сказать, насколько они хороши). Если же попробовать рассмотреть гипотезы по отдельности, то окажется, что лучше, чем плацебо, окажется еще и `Пентобарбитал (150 мг)	`, у которого нескоректированное p-value будет ровно $0.05.$ Также отмечу, что не построил сводную таблицу со всеми методами, так как она была бы слишком объемной и неудобной: в ней было бы слишком много столбцов.